In [ ]:
import tensorflow as tf
import os
import glob
import cv2
import numpy as np

In [ ]:
path = "DATASET/"
train = tf.keras.preprocessing.image_dataset_from_directory(
  path,
  labels = "inferred",
  label_mode = "categorical",
  color_mode = "rgb",
  batch_size = 32,
  image_size = (160,160),
  seed = 32,
  subset = "training",
  validation_split = 0.2  
)
valid = tf.keras.preprocessing.image_dataset_from_directory(
  path,
  labels = "inferred",
  label_mode = "categorical",
  color_mode = "rgb",
  batch_size = 32,
  image_size = (160,160),
  seed = 1234,
  subset = "validation",
  validation_split = 0.2
)

In [ ]:
train

classnames = train.class_names
classes = len(classnames)
classnames

In [ ]:
model = tf.keras.models.Sequential([
tf.keras.layers.Rescaling(1./255, input_shape=(160,160, 3)),
tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
tf.keras.layers.MaxPooling2D(),
tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
tf.keras.layers.MaxPooling2D(),
tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
tf.keras.layers.MaxPooling2D(),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dense(classes, activation='softmax') 
])
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.fit(train, validation_data = valid, epochs = 5, batch_size = 12)

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

cap = cv2.VideoCapture(0)
while cap.isOpened():
    _, img = cap.read()
    #img = cv2.rotate(img, cv2.ROTATE_180)
    if _:
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = faceCascade.detectMultiScale(rgb, 1.9, 5)
        
        for (x,y,w,h) in faces:
            imgROI = rgb[y:y+h, x:x+w]
            imgResize = cv2.resize(imgROI, (160,160))

            img_array = tf.keras.utils.img_to_array(imgResize)
            img_array = tf.expand_dims(img_array, 0)

            predictions = model.predict(img_array)
            score = tf.nn.softmax(predictions[0])

            text = "ID: {}, {:.2f}%.".format(classnames[np.argmax(score)], 100 * np.max(score))

            image = cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,255), 1)
            cv2.putText(image, text, (x, y-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.9, (0,255,255), 1)
            cv2.imshow("Camera", img)
    else:
       print('No Camera')
       cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
id = input("ID")
ids = str(id)

from glob import glob

for image in glob("IMAGES/" + ids + "/*.jpg"):
    img = cv2.imread(image)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = faceCascade.detectMultiScale(rgb, 1.9, 5)

    for (x,y,w,h) in faces:
        imgROI = rgb[y:y+h, x:x+w]
        imgResize = cv2.resize(imgROI, (160,160))
        img_array = tf.keras.utils.img_to_array(imgResize)
        img_array = tf.expand_dims(img_array, 0)
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        text = "ID: {}, {:.2f}%.".format(classnames[np.argmax(score)], 100 * np.max(score))
        print (text)

        cv2.imwrite("file.jpg", imgResize)